In [1]:
import xgboost as xgb
print(xgb.__version__)

3.0.4


In [2]:
import sys, xgboost as xgb
print(sys.executable)        # should point to .../.venv/bin/python
print(xgb.__version__)       # should print 3.0.4
print(xgb.__file__)          # should live under .../.venv/...

/Users/riadanas/Desktop/housing regression MLE/.venv/bin/python
3.0.4
/Users/riadanas/Desktop/housing regression MLE/.venv/lib/python3.11/site-packages/xgboost/__init__.py


In [1]:
# ==============================================
# 1. Imports
# ==============================================
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from xgboost import XGBRegressor
import optuna
import mlflow
import mlflow.xgboost

/Users/sebo/Projects/Regression_ML_EndtoEnd/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# ==============================================
# 2. Load processed datasets
# ==============================================
train_df = pd.read_csv("../data/processed/feature_engineered_train.csv")
eval_df  = pd.read_csv("../data/processed/feature_engineered_eval.csv")


# Define target + features
target = "price"
X_train, y_train = train_df.drop(columns=[target]), train_df[target]
X_eval, y_eval   = eval_df.drop(columns=[target]), eval_df[target]

print("Train shape:", X_train.shape)
print("Eval shape:", X_eval.shape)

Train shape: (576815, 39)
Eval shape: (148448, 39)


In [3]:
# ==============================================
# 3. Define Optuna objective function with MLflow
# ==============================================
def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 200, 1000),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        "gamma": trial.suggest_float("gamma", 0.0, 5.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 10.0, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 10.0, log=True),
        "random_state": 42,
        "n_jobs": -1,
        "tree_method": "hist",
    }

    with mlflow.start_run(nested=True):
        model = XGBRegressor(**params)
        model.fit(X_train, y_train)

        y_pred = model.predict(X_eval)
        rmse = float(np.sqrt(mean_squared_error(y_eval, y_pred)))
        mae = float(mean_absolute_error(y_eval, y_pred))
        r2 = float(r2_score(y_eval, y_pred))

        # Log hyperparameters + metrics
        mlflow.log_params(params)
        mlflow.log_metrics({"rmse": rmse, "mae": mae, "r2": r2})

    return rmse

In [4]:
# ==============================================
# 4. Run Optuna study with MLflow
# ==============================================
# Force MLflow to always use the root project mlruns folder
mlflow.set_tracking_uri("../mlruns")
mlflow.set_experiment("xgboost_optuna_housing")

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=15)

print("Best params:", study.best_trial.params)

2025/10/27 15:55:10 INFO mlflow.tracking.fluent: Experiment with name 'xgboost_optuna_housing' does not exist. Creating a new experiment.
[I 2025-10-27 15:55:10,618] A new study created in memory with name: no-name-657b2d23-ec7e-4c4c-b0a3-e11637dc9363
[I 2025-10-27 15:55:23,550] Trial 0 finished with value: 78947.02731429378 and parameters: {'n_estimators': 927, 'max_depth': 9, 'learning_rate': 0.2773386225606036, 'subsample': 0.9389982069774979, 'colsample_bytree': 0.7859775643402984, 'min_child_weight': 6, 'gamma': 4.412620585320625, 'reg_alpha': 5.904712187951505, 'reg_lambda': 0.5799042287453137}. Best is trial 0 with value: 78947.02731429378.
[I 2025-10-27 15:55:26,678] Trial 1 finished with value: 76835.37370573603 and parameters: {'n_estimators': 605, 'max_depth': 3, 'learning_rate': 0.07723617393568023, 'subsample': 0.6259314782302738, 'colsample_bytree': 0.6537782449063105, 'min_child_weight': 6, 'gamma': 1.3697215055254448, 'reg_alpha': 3.2896777024647956e-06, 'reg_lambda': 8

Best params: {'n_estimators': 428, 'max_depth': 7, 'learning_rate': 0.06887633522014171, 'subsample': 0.9923752265973927, 'colsample_bytree': 0.6291087711786132, 'min_child_weight': 4, 'gamma': 2.759574642692789, 'reg_alpha': 1.1601196098114817e-06, 'reg_lambda': 0.020843543082889604}


In [5]:
# ==============================================
# 5. Train final model with best params and log to MLflow
# ==============================================
best_params = study.best_trial.params
best_model = XGBRegressor(**best_params)
best_model.fit(X_train, y_train)

y_pred = best_model.predict(X_eval)

mae = mean_absolute_error(y_eval, y_pred)
rmse = np.sqrt(mean_squared_error(y_eval, y_pred))
r2 = r2_score(y_eval, y_pred)

print("Final tuned model performance:")
print("MAE:", mae)
print("RMSE:", rmse)
print("R²:", r2)

# Log final model
with mlflow.start_run(run_name="best_xgboost_model"):
    mlflow.log_params(best_params)
    mlflow.log_metrics({"rmse": rmse, "mae": mae, "r2": r2})
    mlflow.xgboost.log_model(best_model, name="model")

Final tuned model performance:
MAE: 32113.973916551815
RMSE: 69725.74557413971
R²: 0.9624295527656418


/Users/sebo/Projects/Regression_ML_EndtoEnd/.venv/lib/python3.11/site-packages/xgboost/sklearn.py:1115: UserWarning: [15:56:49] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1575: Saving model in the UBJSON format as default.  You can use a file extension: `json` or `ubj` to choose between formats.
  self.get_booster().save_model(fname)
2025/10/27 15:56:52 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
